20180634_최준혁_HW3

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# The effect of a welfare program on the birth weight

In [2]:
data = pd.read_excel('lowbirth.xls')
df = data.copy()
ids = df.id.unique()
df["C"] = df["id"] #id를 time invariant variable c의 역할로 넣음
df["ID"]= df["id"]
df.head()

,id,year,lowbrth,infmort,afdcprt,popul,pcinc,physic,afdcprc,d90,...,cpovrate,afdcpsq,cafdcpsq,physicpc,lphypc,clphypc,lpopul,clpopul,C,ID
0,1,1987,8.0,12.2,132,4084,12039,151,3.232125,0,...,NaN,10.446634,NaN,0.036974,-3.297552,NaN,8.314832,NaN,1,1
1,1,1990,8.4,10.8,132,4041,14899,158,3.266518,1,...,-2.099998,10.670140,0.223506,0.039099,-3.241652,0.055900,8.304248,-0.010584,1,1
2,2,1987,4.8,10.4,19,524,18461,138,3.625954,0,...,NaN,13.147544,NaN,0.263359,-1.334238,NaN,6.261492,NaN,2,2
3,2,1990,4.8,10.5,24,550,20867,146,4.363636,1,...,-0.600000,19.041323,5.893779,0.265455,-1.326312,0.007926,6.309918,0.048427,2,2
4,3,1987,6.4,9.5,91,3400,14322,191,2.676471,0,...,NaN,7.163495,NaN,0.056176,-2.879257,NaN,8.131531,NaN,3,3


## Estimate the regression model in equation (1) by using both fixed and random effects models.Which model would you prefer? Explain.

- random effect assumption이 만족하지 않으므로
- 따라서 fixed effect모델을 선호한다

fixed effect

In [3]:
df_fe = df[['lowbrth',"afdcprc",'lpcinc','lphypc','lpopul','d90','C','id']] 

within = pd.DataFrame()
for i in ids:
    temp = df_fe[df_fe.id==i] - df_fe[df_fe.id==i].mean() # mean difference
    within = pd.concat([within, temp],axis = 0)
within.drop(columns = ["id"], inplace = True)
y = within['lowbrth']
X = within.iloc[:,1:]
X["const"] = 1 ; X = X[['const',"afdcprc",'lpcinc','lphypc','lpopul','d90','C']]

In [4]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary()) #time invariant variable C는 추정 불가

                            OLS Regression Results                            
Dep. Variable:                lowbrth   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     9.665
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           1.81e-07
Time:                        23:39:52   Log-Likelihood:                 64.608
No. Observations:                 100   AIC:                            -117.2
Df Residuals:                      94   BIC:                            -101.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.469e-18      0.013  -2.65e-16      1.0

random effect

In [5]:
data = pd.read_excel('lowbirth.xls')
df = data.copy()
df["C"] = df['id']
year = pd.Categorical(df.year)
df = df.set_index(["id", "year"])
df["year"] = year
df

lowbrth  infmort  afdcprt  popul  pcinc  physic   afdcprc  d90  \
id year                                                                   
1  1987      8.0     12.2      132   4084  12039     151  3.232125    0   
   1990      8.4     10.8      132   4041  14899     158  3.266518    1   
2  1987      4.8     10.4       19    524  18461     138  3.625954    0   
   1990      4.8     10.5       24    550  20867     146  4.363636    1   
3  1987      6.4      9.5       91   3400  14322     191  2.676471    0   
...          ...      ...      ...    ...    ...     ...       ...  ...   
48 1990      7.1      9.9      109   1793  13722     166  6.079197    1   
49 1987      5.4      8.6      271   4807  14674     181  5.637612    0   
   1990      5.9      8.2      236   4892  17455     189  4.824203    1   
50 1987      7.7      9.2       13    490  12836     134  2.653061    0   
   1990      7.4      8.6       16    454  16467     139  3.524229    1   

           lpcinc  cafdcprc  ...  cpovrate    afdcpsq  cafdcpsq  physicpc  \
id year                      ...                                            
1  1987  9.395906       NaN  ...       NaN  10.446634       NaN  0.036974   
   1990  9.609050  0.034393  ... -2.099998  10.670140  0.223506  0.039099   
2  1987  9.823416       NaN  ...       NaN  13.147544       NaN  0.263359   
   1990  9.945924  0.737682  ... -0.600000  19.041323  5.893779  0.265455   
3  1987  9.569552       NaN  ...       NaN   7.163495       NaN  0.056176   
...           ...       ...  ...       ...        ...       ...       ...   
48 1990  9.526755  0.230936  ... -3.500000  36.956635  2.754475  0.092582   
49 1987  9.593833       NaN  ...       NaN  31.782667       NaN  0.037653   
   1990  9.767382 -0.813409  ...  0.300000  23.272934 -8.509733  0.038635   
50 1987  9.460009       NaN  ...       NaN   7.038733       NaN  0.273469   
   1990  9.709114  0.871168  ...  0.200000  12.420191  5.381457  0.306167   

           lphypc   clphypc    lpopul   clpopul   C  year  
id year                                                    
1  1987 -3.297552       NaN  8.314832       NaN   1  1987  
   1990 -3.241652  0.055900  8.304248 -0.010584   1  1990  
2  1987 -1.334238       NaN  6.261492       NaN   2  1987  
   1990 -1.326312  0.007926  6.309918  0.048427   2  1990  
3  1987 -2.879257       NaN  8.131531       NaN   3  1987  
...           ...       ...       ...       ...  ..   ...  
48 1990 -2.379658  0.069032  7.491645 -0.056911  48  1990  
49 1987 -3.279331       NaN  8.477828       NaN  49  1987  
   1990 -3.253609  0.025722  8.495357  0.017529  49  1990  
50 1987 -1.296566       NaN  6.194406       NaN  50  1987  
   1990 -1.183623  0.112942  6.118097 -0.076308  50  1990  

[100 rows x 37 columns]

In [6]:
df_re = df[["afdcprc",'lpcinc','lphypc','lpopul','d90','C']]
df_re['const'] = 1 ;  df_re = df_re[['const',"afdcprc",'lpcinc','lphypc','lpopul','d90','C']]
df_re

const   afdcprc    lpcinc    lphypc    lpopul  d90   C
id year                                                        
1  1987      1  3.232125  9.395906 -3.297552  8.314832    0   1
   1990      1  3.266518  9.609050 -3.241652  8.304248    1   1
2  1987      1  3.625954  9.823416 -1.334238  6.261492    0   2
   1990      1  4.363636  9.945924 -1.326312  6.309918    1   2
3  1987      1  2.676471  9.569552 -2.879257  8.131531    0   3
...        ...       ...       ...       ...       ...  ...  ..
48 1990      1  6.079197  9.526755 -2.379658  7.491645    1  48
49 1987      1  5.637612  9.593833 -3.279331  8.477828    0  49
   1990      1  4.824203  9.767382 -3.253609  8.495357    1  49
50 1987      1  2.653061  9.460009 -1.296566  6.194406    0  50
   1990      1  3.524229  9.709114 -1.183623  6.118097    1  50

[100 rows x 7 columns]

In [7]:
mod = RandomEffects(df.lowbrth, df_re)
re_res = mod.fit()
re_res

Dep. Variable:,lowbrth,R-squared:,0.2021
Estimator:,RandomEffects,R-squared (Between):,0.2189
No. Observations:,100,R-squared (Within):,0.1868
Date:,"Wed, Nov 30 2022",R-squared (Overall):,0.2183
Time:,23:39:52,Log-likelihood,22.742
Cov. Estimator:,Unadjusted,,
,,F-statistic:,3.9260
Entities:,50,P-value,0.0015
Avg Obs:,2.0000,Distribution:,"F(6,93)"
Min Obs:,2.0000,,
Max Obs:,2.0000,F-statistic (robust):,3.9260


## Interpret the estimated coefficients from the fixed effects model. You do not need to consider gamma

- 5%수준에서 유의한 변수는 afdcprc와 lphypc이며 
- AFDC의 비율이 1%증가할 수록 the proportion of births with low weights가 0.18%감소한다
- physicians의 비율이 1%증가할 수록 the proportion of births with low weights가 4%증가한다

In [8]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                lowbrth   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     9.665
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           1.81e-07
Time:                        23:39:52   Log-Likelihood:                 64.608
No. Observations:                 100   AIC:                            -117.2
Df Residuals:                      94   BIC:                            -101.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.469e-18      0.013  -2.65e-16      1.0

## Assuming that lpinc, lphypc, and lpopul are strictly exogenous, argue about the validity of imposing the strict exogeneity assumption on afdcprc.


ans : afdcprc에 강한 외생성 가정을 부여해도 정당하다
- 나머지 변수들이 강한 외생성조건을 만족하면 에러텀이 zero conditional mean이므로 afdcprc도 강한 외생성을 가진다

## Your friends suggests using standard errors clustered at the state level. Provide a reason for this suggestion. How does the estimation result change when you use clustered standard errors?


ans :
- 우리가 이분산이 있는 데이터에 white robust s.e를 사용하는 것처럼 이 데이터와 같이 그룹 범주가 존재하는 경우에는 그룹간의 분산으로 진행하는 것이 옳다
- 하지만 cluster변수를 사용했더니 신뢰구간이 늘어났다

In [9]:
within = pd.concat([within,data.id],axis = 1)
within = within.dropna()

In [10]:
reg_model = ols("lowbrth ~ afdcprc + lpcinc	 + lphypc +lpopul+ d90+ C", within)
reg_model = reg_model.fit(cov_type='cluster', cov_kwds={'groups': within.id}) #clustered
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                lowbrth   R-squared:                       0.340
Model:                            OLS   Adj. R-squared:                  0.304
Method:                 Least Squares   F-statistic:                     6.564
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           9.54e-05
Time:                        23:39:52   Log-Likelihood:                 64.608
No. Observations:                 100   AIC:                            -117.2
Df Residuals:                      94   BIC:                            -101.6
Df Model:                           5                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.469e-18   3.24e-18     -1.072      0.284   -9.82e-18    2.88e-18
afdcprc       -0.1851      0.074     -2.502      0.012      -0.330      -0.040
lpcinc        -0.9807      1.566     -0.626      0.531      -4.051       2.090
lphypc         4.1048      2.582      1.590      0.112      -0.957       9.166
lpopul         2.6689      2.708      0.986      0.324      -2.638       7.976
d90            0.2761      0.327      0.844      0.399      -0.365       0.917
C                   0          0        nan        nan           0           0
==============================================================================
Omnibus:                        1.300   Durbin-Watson:                   2.685
Prob(Omnibus):                  0.522   Jarque-Bera (JB):                0.793
Skew:                           0.000   Prob(JB):                        0.673
Kurtosis:                       3.436   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# The effect of executions on murders (Wooldridge C.7)

In [11]:
data = pd.read_excel('murder.xls')
df2 = data.copy()
# df2["C"] = df["id"] #id를 time invariant variable c의 역할로 넣음
df2.columns = ['id', 'state', 'year', 'mrdrte', 'exec', 'unem', 'd90', 'd93', 'cmrdrte', 'cexec','cunem','cexec_1','cunem_1']
df2["C"] = df2['id'] ; df2["ID"] = df2['id']
year = pd.Categorical(df2.year)
df2 = df2.set_index(["id", "year"])
df2["year"] = year
df2.year = df2.year.astype(int)
df2['const'] = 0
df2['const'][df2.year == 87] = 1
df2['const'][df2.year == 90] = 2
df2['const'][df2.year == 93] = 3
df2

state  mrdrte  exec  unem  d90  d93 cmrdrte cexec cunem cexec_1  \
id year                                                                   
1  90    "AL"    11.6     5   6.8    1    0     2.3     3    -1       .   
   93    "AL"    11.6     2   7.5    0    1       0    -3   0.7       3   
2  87    "AK"    10.1     0  10.8    0    0       .     .     .       .   
   90    "AK"     7.5     0   6.9    1    0    -2.6     0  -3.9       .   
   93    "AK"     9.0     0   7.6    0    1     1.5     0   0.7       0   
...       ...     ...   ...   ...  ...  ...     ...   ...   ...     ...   
50 90    "WI"     4.6     0   4.4    1    0     1.1     0  -1.7       .   
   93    "WI"     4.4     0   4.7    0    1    -0.2     0   0.3       0   
51 87    "WY"     2.0     0   8.6    0    0       .     .     .       .   
   90    "WY"     4.9     0   5.4    1    0     2.9     0  -3.2       .   
   93    "WY"     3.4     1   5.4    0    1    -1.5     1     0       0   

        cunem_1   C  ID  year  const  
id year                               
1  90         .   1   1    90      2  
   93        -1   1   1    93      3  
2  87         .   2   2    87      1  
   90         .   2   2    90      2  
   93      -3.9   2   2    93      3  
...         ...  ..  ..   ...    ...  
50 90         .  50  50    90      2  
   93      -1.7  50  50    93      3  
51 87         .  51  51    87      1  
   90         .  51  51    90      2  
   93      -3.2  51  51    93      3  

[152 rows x 15 columns]

## If past executions of convicted murderers have a deterrent effect, what should be the sign of β1? What sign do you think β2 should have? Explain.

ans : 유죄 판결을 받은 살인자들의 과거 처형이 억제 효과가 있다면 
- β1은 음수의 값을 가질 것이다. 억제 효과가 있다면 과거 3년 동안 사형을 집행을 많이 할 수록 살인을 덜 할 것이다.
- β2는 양수의 값을 가지거나 유의하지 않을 것이다. 실업률이 올라가면 생계유지가 힘들어지면서 범죄발생이 잦아질 수 있으므로 양수의 값을 가질 수도 있지만 그 범죄가 살인까지 이어지지 않을 가능 성도 높다

## Using just the years 1990 and 1993, estimate the equation from part (i) by pooled OLS. Ignore the serial correlation problem in the composite errors. Do you find any evidence for a deterrent effect?

- exec의 p-value가 높게 나와서 유의하지 않으므로 억제효과가 있다고 볼수 없다
- 그러나 실업률은 유의하게 양수의 값을 가진다

In [12]:
df_2year = df2[df2.year != 87]
df_2year

state  mrdrte  exec  unem  d90  d93 cmrdrte cexec cunem cexec_1  \
id year                                                                   
1  90    "AL"    11.6     5   6.8    1    0     2.3     3    -1       .   
   93    "AL"    11.6     2   7.5    0    1       0    -3   0.7       3   
2  90    "AK"     7.5     0   6.9    1    0    -2.6     0  -3.9       .   
   93    "AK"     9.0     0   7.6    0    1     1.5     0   0.7       0   
3  90    "AZ"     7.7     0   5.3    1    0     0.2     0  -0.9       .   
...       ...     ...   ...   ...  ...  ...     ...   ...   ...     ...   
49 93    "WV"     6.9     0  10.8    0    1     1.2     0   2.5       0   
50 90    "WI"     4.6     0   4.4    1    0     1.1     0  -1.7       .   
   93    "WI"     4.4     0   4.7    0    1    -0.2     0   0.3       0   
51 90    "WY"     4.9     0   5.4    1    0     2.9     0  -3.2       .   
   93    "WY"     3.4     1   5.4    0    1    -1.5     1     0       0   

        cunem_1   C  ID  year  const  
id year                               
1  90         .   1   1    90      2  
   93        -1   1   1    93      3  
2  90         .   2   2    90      2  
   93      -3.9   2   2    93      3  
3  90         .   3   3    90      2  
...         ...  ..  ..   ...    ...  
49 93      -2.5  49  49    93      3  
50 90         .  50  50    90      2  
   93      -1.7  50  50    93      3  
51 90         .  51  51    90      2  
   93      -3.2  51  51    93      3  

[102 rows x 15 columns]

In [13]:
from linearmodels.panel import PooledOLS

exog_vars = ["const", "exec", "unem", "C"]
mod = PooledOLS(df_2year.mrdrte, df_2year[exog_vars])
pooled_res = mod.fit()
print(pooled_res)

                          PooledOLS Estimation Summary                          
Dep. Variable:                 mrdrte   R-squared:                        0.4794
Estimator:                  PooledOLS   R-squared (Between):              0.4905
No. Observations:                 102   R-squared (Within):              -5.7630
Date:                Wed, Nov 30 2022   R-squared (Overall):              0.4794
Time:                        23:39:52   Log-likelihood                   -377.80
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      22.563
Entities:                          51   P-value                           0.0000
Avg Obs:                       2.0000   Distribution:                    F(4,98)
Min Obs:                       2.0000                                           
Max Obs:                       2.0000   F-statistic (robust):             22.563
                            

##  Now, using 1990 and 1993, estimate the equation by fixed effects. You may use first differencing since you are only using two years of data. Is there evidence of a deterrent effect? How strong? For this question, report standard errors clustered at the state level.

- exec의 p-vallue가 0.17수준으로 높다. 억제효과는 있다고 볼 수 없다

In [14]:
df_2year.drop(columns = "state", inplace = True)
df_2year.drop(columns =['cexec','cunem','cexec_1','cunem_1','cmrdrte'],inplace = True)
df_90 = df_2year[df_2year.year == 90]
df_93 = df_2year[df_2year.year == 93]
display(df_90) ; display(df_93)

,,mrdrte,exec,unem,d90,d93,C,ID,year,const
id,year,,,,,,,,,
1,90,11.6,5,6.8,1,0,1,1,90,2
2,90,7.5,0,6.9,1,0,2,2,90,2
3,90,7.7,0,5.3,1,0,3,3,90,2
4,90,10.3,2,6.9,1,0,4,4,90,2
5,90,11.9,0,5.6,1,0,5,5,90,2
6,90,4.2,0,4.9,1,0,6,6,90,2
7,90,5.1,0,5.1,1,0,7,7,90,2
8,90,5.0,0,5.1,1,0,8,8,90,2
9,90,77.8,0,6.6,1,0,9,9,90,2


,,mrdrte,exec,unem,d90,d93,C,ID,year,const
id,year,,,,,,,,,
1,93,11.6,2,7.5,0,1,1,1,93,3
2,93,9.0,0,7.6,0,1,2,2,93,3
3,93,8.6,3,6.2,0,1,3,3,93,3
4,93,10.2,2,6.2,0,1,4,4,93,3
5,93,13.1,2,9.2,0,1,5,5,93,3
6,93,5.8,0,5.2,0,1,6,6,93,3
7,93,6.3,0,6.2,0,1,7,7,93,3
8,93,5.0,0,5.3,0,1,8,8,93,3
9,93,78.5,0,8.5,0,1,9,9,93,3


In [15]:
df_93 = df_93.reset_index(drop=True) 
df_90 = df_90.reset_index(drop=True) 
df_FD = df_93 - df_90

In [16]:
df_FD.head()

,mrdrte,exec,unem,d90,d93,C,ID,year,const
0,0.0,-3,0.7,-1,1,0,0,3,1
1,1.5,0,0.7,-1,1,0,0,3,1
2,0.9,3,0.9,-1,1,0,0,3,1
3,-0.1,0,-0.7,-1,1,0,0,3,1
4,1.2,2,3.6,-1,1,0,0,3,1


In [17]:
reg_model = ols("mrdrte ~ const + exec + unem + C", df_2year)
reg_model = reg_model.fit(cov_type='cluster', cov_kwds={'groups': df_2year.ID}) #clustered
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 mrdrte   R-squared:                       0.139
Model:                            OLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     9.652
Date:                Wed, 30 Nov 2022   Prob (F-statistic):           7.29e-06
Time:                        23:39:52   Log-Likelihood:                -377.66
No. Observations:                 102   AIC:                             765.3
Df Residuals:                      97   BIC:                             778.4
Df Model:                           4                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2192      2.824      1.140      0.254      -2.316       8.754
const         -1.9273      1.176     -1.639      0.101      -4.232       0.378
exec           0.1837      0.134      1.373      0.170      -0.079       0.446
unem           2.3325      1.319      1.768      0.077      -0.253       4.918
C             -0.1399      0.094     -1.485      0.138      -0.325       0.045
==============================================================================
Omnibus:                      146.083   Durbin-Watson:                   1.112
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4629.041
Skew:                           5.243   Prob(JB):                         0.00
Kurtosis:                      34.293   Cond. No.                         193.
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

##  Use all three years of data and estimate the model by fixed effects. Discuss the size and statistical significance of the deterrent effect compared with only using 1990 and 1993. For this question, report standard errors clustered at the state level.

- 직전의 first differencing과는 다르게 exec의 p-value가 낮고 우리가 예상한대로 계수가 음수값이 나와 0.05%의 유의 수준으로 본다면 억제효과가 있다고 볼 수 있다

In [18]:
temp = df2[df2.ID == 2].reset_index(drop=True) 
temp = temp[['mrdrte',"const",'exec','unem','C']]

In [19]:
temp.add(-temp.mean(axis = 0),axis = 1)

,mrdrte,const,exec,unem,C
0,1.233333,-1.0,0.0,2.366667,0.0
1,-1.366667,0.0,0.0,-1.533333,0.0
2,0.133333,1.0,0.0,-0.833333,0.0


In [20]:
within = pd.DataFrame()
for i in ids:
    temp = df2[df2.ID==i] - df2[df2.ID==i].mean() # mean difference
    within = pd.concat([within, temp],axis = 0)
within = within[['mrdrte',"const",'exec','unem','C']]
y = within['mrdrte']
X = within.iloc[:,1:]
X = X[["const",'exec','unem','C']]

In [21]:
within = pd.concat([within,df2.ID],axis = 1)
within = within.dropna()
within

mrdrte  const  exec      unem    C  ID
id year                                          
1  90    0.000000   -0.5   1.5 -0.350000  0.0   1
   93    0.000000    0.5  -1.5  0.350000  0.0   1
2  87    1.233333   -1.0   0.0  2.366667  0.0   2
   90   -1.366667    0.0   0.0 -1.533333  0.0   2
   93    0.133333    1.0   0.0 -0.833333  0.0   2
...           ...    ...   ...       ...  ...  ..
49 90   -0.100000    0.0   0.0 -1.666667  0.0  49
   93    1.100000    1.0   0.0  0.833333  0.0  49
50 87   -0.666667   -1.0   0.0  1.033333  0.0  50
   90    0.433333    0.0   0.0 -0.666667  0.0  50
   93    0.233333    1.0   0.0 -0.366667  0.0  50

[149 rows x 6 columns]

In [22]:
reg_model = ols("mrdrte ~ const + exec + unem + C", within)
reg_model = reg_model.fit(cov_type='cluster', cov_kwds={'groups': within.ID}) #clustered
reg_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 mrdrte   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     1.736
Date:                Wed, 30 Nov 2022   Prob (F-statistic):              0.172
Time:                        23:39:53   Log-Likelihood:                -368.32
No. Observations:                 149   AIC:                             744.6
Df Residuals:                     145   BIC:                             756.6
Df Model:                           3                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   -1.11e-16   1.31e-16     -0.847      0.397   -3.68e-16    1.46e-16
const          0.8554      0.445      1.921      0.055      -0.018       1.728
exec          -0.1576      0.078     -2.017      0.044      -0.311      -0.004
unem           0.1219      0.258      0.473      0.636      -0.383       0.627
C                   0          0        nan        nan           0           0
==============================================================================
Omnibus:                      185.261   Durbin-Watson:                   2.249
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20337.562
Skew:                          -4.222   Prob(JB):                         0.00
Kurtosis:                      59.609   Cond. No.                          inf
==============================================================================

Notes:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The smallest eigenvalue is      0. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""